In [1]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('spam.csv', encoding="ISO-8859-1")

In [12]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [13]:
df.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [14]:
columns_to_drop = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']

In [16]:
df = df.drop(columns=columns_to_drop)

In [17]:
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
le = LabelEncoder()

In [21]:
df.v1 = le.fit_transform(df.v1)

In [23]:
df  #0 -> Ham , 1 -> Spam

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [27]:
y = df.v1

In [29]:
np.unique(y, return_counts=True)

(array([0, 1]), array([4825,  747]))

In [74]:
df['v2'][4]

"Nah I don't think he goes to usf, he lives around here though"

In [102]:
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

In [113]:
stops = set(stopwords.words('english'))
stemmer = LancasterStemmer()
lamet = WordNetLemmatizer()

In [114]:
import string
punc = set(string.punctuation)

In [115]:
bad = punc.union(stops)

In [116]:
import re

In [117]:
from nltk.tokenize import word_tokenize

In [144]:
def clean_text(message):
    message = message.lower()
    message = re.sub("[^a-zA-Z]+", " ", message)

    words = word_tokenize(message)
    important_words = set(words)-bad
    
    important_words = [lamet.lemmatize(i) for i in important_words]
    important_words = [stemmer.stem(i) for i in important_words]

    return " ".join(important_words)

In [145]:
clean_text(df['v2'][4])

'nah around usf though lif go think'

In [147]:
df['v2'] = df['v2'].apply(clean_text)

In [148]:
df

,v1,v2
0,0,gre wat n world got buffet crazy jurong am bug...
1,0,wif u lar on ok jok
2,1,c entry text fa fin wkly fre comp receiv may a...
3,0,c u hor say already dun ear
4,0,nah around usf though lif go think
...,...,...
5567,1,tim cal nd u easy pound bt priz per contact na...
5568,0,b hom esplanad going fr
5569,0,pity mood suggest
5570,0,buy interest lik week fre bitch els act u gav ...


In [149]:
from sklearn.feature_extraction.text import CountVectorizer

In [216]:
vect = CountVectorizer(max_features=500, min_df=2)

In [217]:
X = df.v2.values

In [218]:
X.shape

(5572,)

In [219]:
y = df.v1.values

In [220]:
y.shape

(5572,)

In [221]:
vect.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=500, min_df=2,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [222]:
X_mod = vect.transform(X).todense()

In [223]:
X_mod.shape

(5572, 500)

In [224]:
from sklearn.model_selection import train_test_split

In [225]:
X_train, X_test, y_train, y_test = train_test_split(
...     X_mod, y, test_size=0.33, random_state=42)

In [226]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [227]:
mnb = MultinomialNB()
bnb = BernoulliNB()

In [228]:
mnb.fit(X_train, y_train)
bnb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [229]:
print(mnb.score(X_test, y_test))
print(bnb.score(X_test, y_test))

0.9700924415443176
0.9804241435562806


In [230]:
text = "IMPORTANT - You could be entitled up to £3,160 in compensation from mis-sold PPI on a credit card or loan. Please reply PPI for info or STOP to opt out."

In [231]:
test_vec = vect.transform([text.lower()]).todense()

In [232]:
bnb.predict(test_vec)

array([1])

In [233]:
text_2 = "let's meet tomorrow for the meeting"

In [234]:
test_vec_2 = vect.transform([text_2.lower()]).todense()

In [235]:
bnb.predict(test_vec_2)

array([0])